**Problem Statement:**
Backorders are unavoidable, but by anticipating which things will be backordered,
planning can be streamlined at several levels, preventing unexpected strain on
production, logistics, and transportation. ERP systems generate a lot of data (mainly
structured) and also contain a lot of historical data; if this data can be properly utilized, a
predictive model to forecast backorders and plan accordingly can be constructed.
Based on past data from inventories, supply chain, and sales, classify the products as
going into backorder (Yes or No).

**What is Backorder ?** 
* A backorder is an order for a good or service that cannot be filled at the current time due to a lack of available supply. The item may not be held in the company's available inventory but could still be in production, or the company may need to still manufacture more of the product. Backorders are an indication that demand for a company's product outweighs its supply.<br>



**Key Takeaways<br>**
* A backorder is an order for a good or service that cannot be filled immediately because of a lack of available supply. 
* Backorders give insight into a company's inventory management. A manageable backorder with a short turnaround is a net positive, but a large backorder with longer wait times can be problematic.
* Companies with manageable backorders tend to have high demand, while those that can't keep up may lose customers.
* However, backorders allow for a company to maintain lower levels of inventory, have lower risk of obsolesce and theft, and may result in natural marketing for its highly demanded product.
* Popular products in high demand (i.e. next generation gaming consoles or new iterations of cell phones) may experience backorders.
#Data Description <br>
**• sku** - Random product(sku) code <br>
**• national_inv**  - Current inventory level of that sku <br>
**• lead_time** - Transit time for product(if available at source)  <br>
**• in_transit_qty** - Quantity in transit from source <br>
**• forecast_x_month** - Forecast sales for the net 3, 6, 9 months <br>
**• sales_x_month** - Sales quantity for the prior 1, 3, 6, 9 months <br>
**• min_bank** - Minimum recommended amount to stock <br>
**• potential_issue** - Indictor variable noting potential issue with item <br>
**• pieces_past_due** - Parts overdue from source <br>
**• perf_x_months_avg** - Source performance in the last 6 and 12 months <br>
**• local_bo_qty** - Amount of stock orders overdue <br>
**• deck_risk** – Part risk flag<br>
**• oe_constraint** – Part risk flag<br>
**• ppap_risk** – Part risk flag<br>
**• stop_auto_buy** – Part risk flag<br>
**• rev_stop** – Part risk flag<br>
**• went_on_backorder** - Product went on backorder <br>
## Get the Data
Welcome to Backorder Machine Learning Prediction Project !! Our task will be , to predict whether the products will go on backorder or not .

## Download the Data

In [13]:
##imports
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:

##Link to download test dataset is https://drive.google.com/file/d/1ABbfz7MjPLj2taxUxwsWxXgYsH-TnBNV/view?usp=share_link
##Link to download train dataset is https://drive.google.com/file/d/1CTiyE6BZlDK-UNYUuuT8FXszBvuT9luS/view?usp=share_link
Path("")
df_test = pd.read_csv(r"/content/drive/MyDrive/bo project data/dataset/Kaggle_Test_Dataset_v2.csv")
df_train = pd.read_csv(r"/content/drive/MyDrive/bo project data/dataset/Kaggle_Training_Dataset_v2.csv")